# Exploring the Word Corpus and EDA

With the dataset cleaned and truncated, this notebook dives deeper into the actual dialogue and building the word corpus. A large part of that involves examining stop words and making decisions on which words to drop from the data.

In [1]:
# Load in the packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
%matplotlib inline
sns.set_style('darkgrid')

The next few cells set up the data. For more details refer to notebook \#1. The adjusted data set is named `lines_final`.

In [2]:
lines = pd.read_csv('../data/All-seasons.csv')

lines = lines[lines.Season != 'Season']

In [3]:
lines[['Season', 'Episode']] = lines[['Season', 'Episode']].astype('int64')

In [4]:
support_chars = ['Mr. Garrison', 'Chef', 'Sharon',\
                 'Mr. Mackey', 'Gerald', 'Liane', 'Sheila',\
                 'Stephen', 'Ms. Garrison', 'Mrs. Garrison']

lines.loc[lines.Character.isin(support_chars), 'Character'] = 'Support Character'

In [5]:
final_labels = ['Cartman', 'Stan', 'Kyle', 'Butters', 'Randy', 'Support Character']

lines_final = lines[lines.Character.isin(final_labels)]

### Word Corpus

Here the basic corpus is created. Essentially, all the strings of dialogue from the `Line` column are compiled into one list. Then, regular expressions are used to remove the new line figure `\n` from each string.

In [6]:
import re

corpus = lines_final.Line.tolist()

for line in range(len(corpus)):
    corpus[line] = re.sub('\\n', '', corpus[line].rstrip())
    
corpus[:10]

['You guys, you guys! Chef is going away.',
 'Going away? For how long?',
 'Forever.',
 "I'm sorry boys.",
 "Chef said he's been bored, so he joining a group called the Super Adventure Club.",
 'Wow!',
 'Chef?? What kind of questions do you think adventuring around the world is gonna answer?!',
 "What's the meaning of life? Why are we here?",
 "I hope you're making the right choice.",
 "I'm gonna miss him.  I'm gonna miss Chef and I...and I don't know how to tell him!"]

<b>Average length</b><br>
What is the average document length for each line?

In [7]:
np.mean([len(doc.split()) for doc in corpus])

11.160658881931518

### Tokens

With the corpus created, we can now get even more granular and examine the individual words. The words from the corpus are compiled into one list and then converted to lowercase with all punctuation removed. Then a counter is called and all the words are added to an OrderedDict according to their frequency within the corpus.

In [8]:
import string

words = " ".join(corpus).lower()
words = " ".join(word.strip(string.punctuation) for word in words.split())

In [9]:
from collections import Counter, OrderedDict

word_counts = Counter(words.split())

token_dict = OrderedDict(word_counts.most_common())

Looking at the OrderedDict shows which words show up the most, indicating likely stop words.

In [11]:
[(k, v) for k, v in token_dict.items()][:250]

[('you', 12969),
 ('the', 10567),
 ('i', 9867),
 ('to', 9464),
 ('a', 7114),
 ('and', 6104),
 ('it', 5390),
 ('that', 4583),
 ('we', 4574),
 ('is', 4417),
 ('of', 3998),
 ('what', 3974),
 ('this', 3362),
 ('in', 3305),
 ('have', 3125),
 ('my', 3066),
 ('on', 3053),
 ('oh', 3019),
 ('all', 2927),
 ('just', 2855),
 ('do', 2740),
 ('me', 2682),
 ("i'm", 2670),
 ('for', 2543),
 ('no', 2528),
 ("don't", 2515),
 ('are', 2442),
 ('be', 2283),
 ('your', 2264),
 ("it's", 2205),
 ('get', 2166),
 ('yeah', 2094),
 ('but', 1989),
 ('not', 1948),
 ('with', 1946),
 ('know', 1928),
 ('so', 1911),
 ('dude', 1881),
 ('now', 1876),
 ('well', 1871),
 ('go', 1819),
 ('can', 1756),
 ('right', 1692),
 ('out', 1663),
 ('like', 1606),
 ('was', 1596),
 ('gonna', 1579),
 ("that's", 1545),
 ('here', 1544),
 ('kyle', 1492),
 ('guys', 1470),
 ('up', 1459),
 ('about', 1450),
 ("you're", 1402),
 ('he', 1382),
 ('come', 1374),
 ('they', 1294),
 ('okay', 1278),
 ('uh', 1248),
 ('see', 1242),
 ('our', 1239),
 ('how', 12

### Frequency Metrics

Beyond just identifying common words, below are a couple functions to analyze different types of word frequency. The first function compares the total number of times a word appears in the corpus, total frequency, versus the number of documents containing the word, the document frequency.

In [12]:
def freq_compare(word, corpus=corpus):
    '''Takes a word and a word corpus and calculates
    the total word frequency and the number of documents
    containing that word.'''
    
    word = word.lower()
    
    words = " ".join(corpus).lower()
    words = " ".join(word.strip(string.punctuation) for word in words.split())
    
    word_counts = Counter(words.split())
    token_dict = OrderedDict(word_counts.most_common())
    
    total_frequency = token_dict[word]
    
    doc_freq = 0
    
    for line in corpus:
        if word in [token.strip(string.punctuation).lower() for token in line.split()]:
            doc_freq += 1
    
    print('The total frequency of the word \'{}\' is: \t'.format(word), total_frequency)
    print('The number of documents with the word \'{}\': \t'.format(word), doc_freq)

Now a sanity check:

In [13]:
freq_compare('A', ['a', 'a a.', 'b'])

The total frequency of the word 'a' is: 	 3
The number of documents with the word 'a': 	 2


The next function compares the total frequency of a particular word between the different character labels. This is useful for assessing if a particular word has more importance for specific characters. 

In [16]:
def compare_labels(term):
    '''Takes a particular word and calculates
    how often it is used by each character'''
    
    term = term.lower()
    
    count_dict = {'Cartman': 0, 'Stan': 0, 'Kyle': 0, 'Butters': 0,\
                  'Randy': 0, 'Support Character': 0}
    
    for k, v in count_dict.items():
        subset = lines_final[lines_final.Character == k]
        
        corpus = subset.Line.tolist()

        for line in range(len(corpus)):
            corpus[line] = re.sub('\\n', '', corpus[line].rstrip())
        
        words = " ".join(corpus).lower()
        words = " ".join(word.strip(string.punctuation) for word in words.split())
    
        word_counts = Counter(words.split())
        token_dict = OrderedDict(word_counts.most_common())
        
        if term in token_dict.keys():
            count_dict[k] += token_dict[term]
        
        # Now convert to a ratio
        count_dict[k] = round((count_dict[k] / len(subset)), 2)
    
    print('How often the word \'{}\' appears in each class:'.format(term))
    print(count_dict)

<b>Trying the functions: </b><br>
Now that the functions are defined, let's examine a couple words from `token_dict`, the OrderedDict of all words. It helps to keep in mind that there are approximately 36,000 documents in the data, and that Cartman, Stan and Kyle have the majority of the lines.

In [17]:
freq_compare('The')
compare_labels('The')

The total frequency of the word 'the' is: 	 10567
The number of documents with the word 'the': 	 7574
How often the word 'the' appears in each class:
{'Cartman': 0.38, 'Stan': 0.23, 'Kyle': 0.22, 'Butters': 0.26, 'Randy': 0.37, 'Support Character': 0.34}


In [18]:
freq_compare('but')
compare_labels('but')

The total frequency of the word 'but' is: 	 1989
The number of documents with the word 'but': 	 1847
How often the word 'but' appears in each class:
{'Cartman': 0.06, 'Stan': 0.05, 'Kyle': 0.05, 'Butters': 0.08, 'Randy': 0.07, 'Support Character': 0.06}


In [19]:
freq_compare('dude')
compare_labels('dude')

The total frequency of the word 'dude' is: 	 1881
The number of documents with the word 'dude': 	 1833
How often the word 'dude' appears in each class:
{'Cartman': 0.04, 'Stan': 0.11, 'Kyle': 0.09, 'Butters': 0.0, 'Randy': 0.0, 'Support Character': 0.0}


The words 'the' and 'but' are common words and have fairly uniform frequencies. On the other hand, the word 'dude', although nearly as frequent as 'but', is used much more frequently by the first three characters. This shows that it might be a good word to help identify those characters.

<b>Sifting through stopwords</b><br>
With the functions defined, we can now run them over the list of common words to check the frequency comparisons and confirm which ones should be dropped as stopwords. Below, I have defined a small function to combine the two functions from above.

In [20]:
def word_freqs(word):
    freq_compare(word)
    compare_labels(word)

In [125]:
word_freqs("on")

The total frequency of the word 'on' is: 	 3053
The number of documents with the word 'on': 	 2755
How often the word 'on' appears in each class:
{'Cartman': 0.11, 'Stan': 0.08, 'Kyle': 0.07, 'Butters': 0.08, 'Randy': 0.1, 'Support Character': 0.08}


For this process, I chose to manually check each word in order to really dig into the list and compare each one. Having done so, the final list of stopwords is defined below. While none of the words in the list are too surprising, some of the words I chose to not treat as stopwords may be of interest. For example, I chose to keep 'my' and 'mine' because they had slightly inbalanced distributions, which might owe to Cartman's selfish nature. I also chose to keep 'no' and 'not' because those might be useful later when moving beyond a simple bag-of-words model.<br>
<br>
The final list contains 106 words.

In [126]:
stop_words = ['you', 'the', 'i', 'to', 'a', 'and', 'it', 'that',\
              'we', 'is', 'of', 'what', 'this', 'in', 'have', 'all',\
              'just', 'do', 'for', "don't", 'are', 'be', "it's", 'get',\
              'but', 'with', 'know', 'so', 'go', 'can', 'right', 'out',\
              'like', 'was', 'gonna', "that's", 'here', 'up', 'about', \
              "you're", 'he', 'come', 'they', 'okay', 'see', 'our',\
              'how', 'if', 'think', 'at', 'us', "can't", "we're", 'got',\
              'there', 'look', 'did', 'why', 'then', 'him', 'time',\
              'back', 'one', 'going', 'want', 'who', "he's", 'from', \
              'some', 'his', 'will', 'need', 'make', 'take', 'yes',\
              "let's", 'because', 'them', 'has', 'as', "what's",\
              "there's", 'too', 'an', 'when', 'been', 'where', 'or',\
              'were', 'had', "they're", 'her', 'by', 'their', 'those',\
              'she', 'these', 'any', 'into', "we've", 'two','does',\
              'much', 'being', 'am', 'than']

In [127]:
len(stop_words)

106

Another thing I noticed while testing the stopwords is that the frequencies for Kyle and Stan were consistenly lower than the other characters. I don't know the exact reason for this, but it seems to indicate that the dialogue for those two characters is a bit more unique.<br>
Here are a couple examples:

In [128]:
compare_labels('in')

How often the word 'in' appears in each class:
{'Cartman': 0.12, 'Stan': 0.07, 'Kyle': 0.07, 'Butters': 0.09, 'Randy': 0.11, 'Support Character': 0.11}


In [129]:
compare_labels('right')

How often the word 'right' appears in each class:
{'Cartman': 0.06, 'Stan': 0.03, 'Kyle': 0.03, 'Butters': 0.04, 'Randy': 0.06, 'Support Character': 0.06}


The frequency proportions are lowere for Kyle and Stan. This type of pattern was surprisingly consistent throughout the list of stopwords